In [ ]:
import PIL.Image as Image

# 1. Choose two large prime numbers p and q
p = 61
q = 53
n = p * q

# 2. Choose a number e that is relatively prime to (p-1)(q-1)
phi = (p - 1) * (q - 1)
e = 17  # 17 is a common choice for e
public_key = (e, n)
print("Public Key:", public_key)

# 3. Find a number d such that de = 1 mod ((p-1)(q-1))
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

def mod_inverse(e, phi):
    for d in range(1, phi):
        if (e * d) % phi == 1:
            return d
    return None

d = mod_inverse(e, phi)
private_key = (d, n)
print("Private Key:", private_key)

# Helper function to get the position of a character in the alphabet
def char_to_pos(char):
    if 'A' <= char <= 'Z':
        return ord(char) - ord('A') + 1
    elif 'a' <= char <= 'z':
        return ord(char) - ord('a') + 27
    else:
        raise ValueError("Character not in alphabet")

# Helper function to get the character from its position in the alphabet
def pos_to_char(pos):
    if 1 <= pos <= 26:
        return chr(pos + ord('A') - 1)
    elif 27 <= pos <= 52:
        return chr(pos + ord('a') - 27)
    else:
        raise ValueError("Position out of range")

# RSA encryption function
def rsa_encrypt(plaintext, public_key):
    e, n = public_key
    ciphertext = [pow(char_to_pos(char), e, n) for char in plaintext]
    return ciphertext

# RSA decryption function
def rsa_decrypt(ciphertext, private_key):
    d, n = private_key
    plaintext = ''.join([pos_to_char(pow(char, d, n)) for char in ciphertext])
    return plaintext

# Example plaintext
plaintext = "ChristopherSpadavecchia"

# Encrypt the plaintext
ciphertext = rsa_encrypt(plaintext, public_key)
print("Ciphertext:", ciphertext)

# Decrypt the ciphertext
decrypted_text = rsa_decrypt(ciphertext, private_key)
print("Decrypted Text:", decrypted_text)

# Steganography part
ENCODED_BIT = 1 - 1

# 1. Convert the string to binary
binary = [bin(char_to_pos(char))[2:].zfill(8) for char in plaintext]
binary.append("00000000")
binary_string = "".join(binary)

# 2. Load the image
image = Image.open("input.jpg")
pixels = list(image.getdata())

# 3. Encode the binary string into the image using the least significant bit of each color channel (RGB)
for i in range(len(binary_string)):
    index = 250000 + i // 3
    pixel_index = i % 3
    
    pixel = list(pixels[index])
    if binary_string[i] == "1":
        pixel[pixel_index] |= (1 << ENCODED_BIT)
    else:
        pixel[pixel_index] &= ~(1 << ENCODED_BIT)

    pixels[index] = tuple(pixel)

# 4. Save the image
image.putdata(pixels)
image.save("output.png")

# 5. Decode the image
binary = []
index = 250000
digit = ""
while True:
    for i in range(3):
        pixel = pixels[index]
        digit += bin(pixel[i])[2:].zfill(8)[7 - ENCODED_BIT]
        if len(digit) == 8:
            if digit == "00000000":
                break
            else:
                binary.append(digit)
                digit = ""

    index += 1
    if digit == "00000000":
        break

# 6. Convert the binary string to a string
string = "".join([pos_to_char(int(byte, 2)) for byte in binary])
print(string)

Public Key: (17, 3233)
Private Key: (2753, 3233)
Ciphertext: [641, 2170, 2412, 3179, 1230, 884, 2185, 612, 2170, 1313, 2412, 1992, 2680, 612, 1632, 1773, 1632, 2578, 1313, 281, 281, 2170, 3179, 1632]
Decrypted Text: Christopher Spadavecchia
Christopher Spadavecchia
